# FIFA 19 Player Undervaluation Analysis

* **Student:** Shola Lajuwomi
* **Class:** AI

## Project Description

Develop a Jupyter Notebook (for Kaggle/GitHub) that performs unsupervised learning (DBSCAN) on the FIFA 19 player dataset to identify potentially undervalued player archetypes within specific positions (e.g., Strikers). The analysis focuses on clustering players based on position-specific composite skill metrics versus their market value ('Value'). The notebook will follow the structure required for the AI class assignment, including introduction, EDA, preprocessing, model training/tuning, and conclusion.

## Target Audience

-   AI Course Instructor (Mr. Dole) for assignment evaluation.
-   CS students interested in a practical unsupervised learning example.
-   Football analysts/enthusiasts interested in player valuation methods.

## Data Source

The analysis uses the "FIFA 19 Complete Player Dataset" available on Kaggle:
[https://www.kaggle.com/datasets/javagarm/fifa-19-complete-player-dataset](https://www.kaggle.com/datasets/javagarm/fifa-19-complete-player-dataset)

## 1. Introduction

**Problem:** Identifying potentially undervalued player archetypes in the FIFA 19 dataset using unsupervised learning. Standard valuation methods might overlook players who offer high skill relative to their market price.

**Goal:** To cluster players within selected positions (starting with Strikers 'ST') based on a composite skill score versus their market value ('Value_EUR'). The aim is to use DBSCAN to identify distinct groups, particularly focusing on clusters representing high-skill, low-value players ("undervalued archetypes").

**Data Source:** The analysis utilizes the "FIFA 19 Complete Player Dataset" sourced from Kaggle.
* Dataset Link: [https://www.kaggle.com/datasets/stefanoleone992/fifa-19-complete-player-dataset](https://www.kaggle.com/datasets/stefanoleone992/fifa-19-complete-player-dataset)
*(Note: This URL is specified in the technical specification, though the README might reference another URL. This analysis will proceed using the specification URL.)*

**Methodology Outline:**
1.  **Data Loading & Initial Exploration:** Load the dataset and perform preliminary checks.
2.  **Data Preprocessing:** Clean the data, handle missing values, convert data types (e.g., height, weight, currency).
3.  **Feature Engineering:** Define and calculate a composite skill score for the target position(s).
4.  **Exploratory Data Analysis (EDA):** Visualize distributions and relationships in the cleaned data and engineered features.
5.  **Feature Scaling:** Scale the selected features (skill score, value) for clustering.
6.  **Unsupervised Learning (DBSCAN):** Apply DBSCAN to the scaled data, including hyperparameter tuning.
7.  **Results Analysis:** Visualize clusters, identify undervalued groups, and examine sample players.
8.  **Conclusion:** Summarize findings, evaluate the model, discuss limitations, and suggest future work.

In [7]:
#This cell imports necessary libraries and sets up the plotting environment.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
# Ensure plots are displayed inline in the notebook
%matplotlib inline
# Set a visually appealing style for the plots
sns.set_style('whitegrid')

print("Libraries imported successfully.")

Libraries imported successfully.


In [8]:
"""## 2. Data Loading and Initial Exploration

This section focuses on loading the dataset from the specified CSV file into a pandas DataFrame and performing initial checks to understand its structure, data types, and basic statistics.
"""

# Define the path to the dataset file
# Assumes the CSV file is in the same directory as the notebook, or in a './data/' subdirectory.
# Update this path if your file is located elsewhere.
data_path = 'FIFA_19_COMPLETE_PLAYER_DATASET.csv'
# data_path = './data/FIFA_19_COMPLETE_PLAYER_DATASET.csv' # Alternative if in a 'data' subfolder

# Attempt to load the dataset
try:
    fifa_df = pd.read_csv(data_path, encoding='Windows-1252')
    print(f"Dataset loaded successfully from '{data_path}'.")
except FileNotFoundError:
    print(f"Error: The file '{data_path}' was not found.")
    print("Please ensure the 'FIFA_19_COMPLETE_PLAYER_DATASET.csv' file is in the correct directory.")
    # Depending on the environment, you might want to stop execution here
    # For example, in a script: import sys; sys.exit()
    # In a notebook, this message serves as a clear warning.
    fifa_df = None # Set to None if loading failed

# Display the first few rows if the dataframe loaded successfully
if fifa_df is not None:
    print("Displaying the first 5 rows of the dataset:")
    # display(fifa_df.head()) # Use display() in Jupyter/Colab for better formatting
    display(fifa_df.head())
else:
    print("Cannot display head because the dataset failed to load.")

Dataset loaded successfully from 'FIFA_19_COMPLETE_PLAYER_DATASET.csv'.
Displaying the first 5 rows of the dataset:


,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31.0,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94.0,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33.0,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94.0,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26.0,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92.0,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27.0,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91.0,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27.0,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91.0,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


### 2.1 DataFrame Info

Let's start by getting a concise summary of the DataFrame. The `.info()` method provides information about the column data types, the number of non-null values, and memory usage. This helps identify columns that might need type conversion or missing value handling.

In [9]:
display(fifa_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18207 non-null  int64  
 1   ID                        18207 non-null  int64  
 2   Name                      18207 non-null  object 
 3   Age                       18206 non-null  float64
 4   Photo                     18207 non-null  object 
 5   Nationality               18207 non-null  object 
 6   Flag                      18207 non-null  object 
 7   Overall                   18206 non-null  float64
 8   Potential                 18207 non-null  int64  
 9   Club                      17966 non-null  object 
 10  Club Logo                 18207 non-null  object 
 11  Value                     18207 non-null  object 
 12  Wage                      18207 non-null  object 
 13  Special                   18207 non-null  int64  
 14  Prefer

None

### 2.2 First Few Rows

Displaying the first few rows using `.head()` gives us a glimpse of the actual data values and helps verify that the data loaded correctly.

In [10]:
display(fifa_df.head())


,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31.0,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94.0,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33.0,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94.0,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26.0,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92.0,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27.0,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91.0,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27.0,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91.0,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


### 2.3 Dataset Shape

Checking the `.shape` attribute tells us the total number of rows (players) and columns (features) in the dataset.

In [11]:
print(f"Dataset shape: {fifa_df.shape}")

Dataset shape: (18207, 89)


### 2.4 Summary Statistics

The `.describe(include='all')` method provides summary statistics for all columns. For numerical columns, it includes count, mean, standard deviation, min, max, and quartiles. For object/categorical columns, it includes count, unique values, top value, and frequency of the top value. This gives a broad overview of the data distribution and potential issues like outliers or skewed distributions.

In [12]:
display(fifa_df.describe(include='all'))

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
count,18207.000000,18207.000000,18207,18206.000000,18207,18207,18207,18206.000000,18207.000000,17966,...,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,16643
unique,NaN,NaN,17194,NaN,18207,164,164,NaN,NaN,651,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1244
top,NaN,NaN,J. Rodríguez,NaN,https://cdn.sofifa.org/players/4/19/246269.png,England,https://cdn.sofifa.org/flags/14.png,NaN,NaN,RC Celta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€1.1M
freq,NaN,NaN,11,NaN,1,1662,1662,NaN,NaN,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557
mean,9103.000000,214298.338606,NaN,25.122048,NaN,NaN,NaN,66.237449,71.307299,NaN,...,58.648274,47.281623,47.697836,45.661435,16.616223,16.391596,16.232061,16.388898,16.710887,NaN
std,5256.052511,29965.244204,NaN,4.670022,NaN,NaN,NaN,6.907059,6.136496,NaN,...,11.436133,19.904397,21.664004,21.289135,17.695349,16.906900,16.502864,17.034669,17.955119,NaN
min,0.000000,16.000000,NaN,16.000000,NaN,NaN,NaN,46.000000,48.000000,NaN,...,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN
25%,4551.500000,200315.500000,NaN,21.000000,NaN,NaN,NaN,62.000000,67.000000,NaN,...,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000,NaN
50%,9103.000000,221759.000000,NaN,25.000000,NaN,NaN,NaN,66.000000,71.000000,NaN,...,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,NaN
75%,13654.500000,236529.500000,NaN,28.000000,NaN,NaN,NaN,71.000000,75.000000,NaN,...,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000,NaN


### 2.5 Check for Duplicate Rows

Duplicate rows can unnecessarily inflate the dataset size and potentially bias analysis results. We check for and count any fully duplicate rows.

In [13]:
duplicate_count = fifa_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


## 3. Data Preprocessing

This section focuses on cleaning and transforming the raw data into a format suitable for analysis and modeling. This involves handling missing values, converting data types, and removing irrelevant information.

### 3.1 Drop Unnecessary Columns

Several columns identified in the project specification are not relevant for this analysis (e.g., URLs, IDs, administrative details) and can be removed.

In [14]:
# List of columns to drop
columns_to_drop = [
    'Unnamed: 0', 'Photo', 'Flag', 'Club Logo', 'Real Face',
    'Joined', 'Loaned From', 'Contract Valid Until', 'Special'
]

# Record shape before dropping
shape_before_drop = fifa_df.shape
print(f"Shape before dropping columns: {shape_before_drop}")

# Drop the columns, using errors='ignore' in case some are already missing
fifa_df = fifa_df.drop(columns=columns_to_drop, errors='ignore')

# Print the new shape
print(f"Shape after dropping columns: {fifa_df.shape}")

# Display the first few rows with remaining columns
display(fifa_df.head())

Shape before dropping columns: (18207, 89)
Shape after dropping columns: (18207, 80)


,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31.0,Argentina,94.0,94,FC Barcelona,€110.5M,€565K,Left,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33.0,Portugal,94.0,94,Juventus,€77M,€405K,Right,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26.0,Brazil,92.0,93,Paris Saint-Germain,€118.5M,€290K,Right,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27.0,Spain,91.0,93,Manchester United,€72M,€260K,Right,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27.0,Belgium,91.0,92,Manchester City,€102M,€355K,Right,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


### 3.2 Handle Missing Position Values

The 'Position' column is critical for our analysis, especially for filtering players and potentially for calculating position-specific skill scores later. Rows where this information is missing cannot be effectively used in the core part of our analysis, so we will drop them.

In [15]:
# Store shape before dropping
shape_before = fifa_df.shape[0]

# Drop rows where 'Position' is NaN
fifa_df.dropna(subset=['Position'], inplace=True)

# Calculate and print the number of rows dropped
rows_dropped = shape_before - fifa_df.shape[0]
print(f"Dropped {rows_dropped} rows with missing 'Position'.")
print(f"New shape: {fifa_df.shape}")

Dropped 60 rows with missing 'Position'.
New shape: (18147, 80)


### 3.3 Impute Missing Club Values

Missing 'Club' information might indicate a player is a free agent or simply missing data. For consistency and to avoid issues with potential grouping or analysis based on clubs later, we will impute these missing values with the string 'No Club'.

In [16]:
# Impute missing 'Club' values
club_nan_before = fifa_df['Club'].isnull().sum()
print(f"Missing 'Club' values before imputation: {club_nan_before}")

fifa_df['Club'].fillna('No Club', inplace=True)

club_nan_after = fifa_df['Club'].isnull().sum()
print(f"Missing 'Club' values after imputation: {club_nan_after}")

# Display the value counts to see 'No Club'
print("\nValue counts for 'Club' column (showing top 5 and 'No Club'):")
display(fifa_df['Club'].value_counts().head())
if 'No Club' in fifa_df['Club'].value_counts():
    print(f"\nCount for 'No Club': {fifa_df['Club'].value_counts()['No Club']}")

Missing 'Club' values before imputation: 229
Missing 'Club' values after imputation: 0

Value counts for 'Club' column (showing top 5 and 'No Club'):


C:\Users\samst\AppData\Local\Temp\ipykernel_21908\1160259138.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fifa_df['Club'].fillna('No Club', inplace=True)


Club
No Club              229
RC Celta              33
Borussia Dortmund     33
Valencia CF           33
AS Monaco             33
Name: count, dtype: int64


Count for 'No Club': 229


### 3.4 Convert Height to Centimeters
The 'Height' column is currently stored as a string (e.g., "5'7"). We need to convert this into a numerical format, specifically centimeters (cm), for calculations and analysis.

We'll define a function to parse the feet and inches from the string, convert them to centimeters (1 foot = 30.48 cm, 1 inch = 2.54 cm), and sum them up. The function will handle potential errors like missing values (NaN) or unexpected string formats, returning NaN in those cases. Finally, we'll apply this function to create a new 'Height_cm' column and drop the original 'Height' column.

In [17]:
# Define the conversion function for height
def convert_height(height_str):
    # Check if the input is a string
    if isinstance(height_str, str):
        # Use regex to find feet and inches pattern (e.g., 5'7)
        match = re.match(r"(\d)'(\d{1,2})", height_str)
        if match:
            try:
                feet = int(match.group(1))
                inches = int(match.group(2))
                # Convert to centimeters: (feet * 30.48) + (inches * 2.54)
                height_cm = (feet * 30.48) + (inches * 2.54)
                return height_cm
            except ValueError:
                # Handle cases where conversion to int fails (though regex should prevent this)
                return np.nan
        else:
            # Return NaN if the string doesn't match the expected format
            return np.nan
    # Return NaN for non-string inputs (like NaN itself)
    return np.nan

# Apply the function to the 'Height' column
fifa_df['Height_cm'] = fifa_df['Height'].apply(convert_height)

# Check for NaNs created during conversion
nan_height_count = fifa_df['Height_cm'].isnull().sum()
print(f"NaNs created in Height_cm during conversion: {nan_height_count}")

# Drop the original 'Height' column
fifa_df.drop(columns=['Height'], inplace=True)

# Display the head of the new column and some basic info
print("\nHead of the new 'Height_cm' column:")
display(fifa_df[['Name', 'Height_cm']].head())
print("\nDescription of 'Height_cm':")
display(fifa_df['Height_cm'].describe())

NaNs created in Height_cm during conversion: 0

Head of the new 'Height_cm' column:


,Name,Height_cm
0,L. Messi,170.18
1,Cristiano Ronaldo,187.96
2,Neymar Jr,175.26
3,De Gea,193.04
4,K. De Bruyne,180.34



Description of 'Height_cm':


count    18147.000000
mean       181.259450
std          6.730335
min        154.940000
25%        175.260000
50%        180.340000
75%        185.420000
max        205.740000
Name: Height_cm, dtype: float64

### 3.5 Convert Weight to Kilograms

Similarly, the 'Weight' column is stored as a string (e.g., '159lbs'). We need to convert this to a numerical value in kilograms (kg) for analysis. We'll extract the numerical part and apply the conversion factor (1 lbs ≈ 0.453592 kg). Invalid or missing values will be handled by returning NaN.

In [18]:
def convert_weight(weight_str):
    """
    Converts a weight string (e.g., '159lbs') to kilograms (float).

    Args:
        weight_str (str or float): The weight string or potentially NaN.

    Returns:
        float: Weight in kilograms, or NaN if input is invalid/NaN.
    """
    if pd.isna(weight_str) or not isinstance(weight_str, str):
        return np.nan
    match = re.match(r"(\d+)lbs", weight_str)
    if match:
        try:
            pounds = int(match.group(1))
            kg = pounds * 0.453592
            return float(kg)
        except (ValueError, TypeError):
            return np.nan
    return np.nan

# Apply the function to create the 'Weight_kg' column
fifa_df['Weight_kg'] = fifa_df['Weight'].apply(convert_weight)

# Drop the original 'Weight' column
fifa_df.drop(columns=['Weight'], inplace=True)

# Check for NaNs created and display head
print(f"NaNs in Weight_kg: {fifa_df['Weight_kg'].isnull().sum()}")
display(fifa_df[['Name', 'Weight_kg']].head())

NaNs in Weight_kg: 0


,Name,Weight_kg
0,L. Messi,72.121128
1,Cristiano Ronaldo,83.007336
2,Neymar Jr,68.038800
3,De Gea,76.203456
4,K. De Bruyne,69.853168


### 3.6 Convert Value and Wage to Numerical

The `Value` and `Wage` columns store monetary values as strings with currency symbols ('€') and suffixes ('M' for millions, 'K' for thousands). We need to convert these into numerical representations (float) to perform calculations. We will define a function to handle this conversion, removing the '€' symbol, interpreting the 'M' and 'K' suffixes, and converting the result to a float. Invalid formats or missing values will be converted to NaN.

In [19]:
# Define the conversion function
def convert_currency(currency_str):
    """
    Converts a currency string (e.g., '€110.5M', '€565K', '€0') to a float value.

    Args:
        currency_str (str): The currency string to convert.

    Returns:
        float or np.nan: The numerical value in EUR, or NaN if conversion fails or input is invalid.
    """
    if pd.isna(currency_str) or not isinstance(currency_str, str):
        return np.nan

    currency_str = currency_str.replace('€', '').strip() # Remove Euro symbol and strip whitespace
    multiplier = 1.0

    if 'M' in currency_str:
        multiplier = 1_000_000.0
        currency_str = currency_str.replace('M', '')
    elif 'K' in currency_str:
        multiplier = 1_000.0
        currency_str = currency_str.replace('K', '')

    try:
        value = float(currency_str) * multiplier
        return value
    except ValueError:
        # Return NaN if conversion to float fails
        return np.nan

# Apply the function to 'Value' and 'Wage' columns
fifa_df['Value_EUR'] = fifa_df['Value'].apply(convert_currency)
fifa_df['Wage_EUR'] = fifa_df['Wage'].apply(convert_currency)

# Drop the original columns
fifa_df.drop(columns=['Value', 'Wage'], inplace=True)

# Check for NaNs created and display head
print(f"NaNs in Value_EUR: {fifa_df['Value_EUR'].isnull().sum()}")
print(f"NaNs in Wage_EUR: {fifa_df['Wage_EUR'].isnull().sum()}")
print("\nHead of new Value_EUR and Wage_EUR columns:")
display(fifa_df[['Name', 'Value_EUR', 'Wage_EUR']].head())

NaNs in Value_EUR: 0
NaNs in Wage_EUR: 0

Head of new Value_EUR and Wage_EUR columns:


,Name,Value_EUR,Wage_EUR
0,L. Messi,110500000.0,565000.0
1,Cristiano Ronaldo,77000000.0,405000.0
2,Neymar Jr,118500000.0,290000.0
3,De Gea,72000000.0,260000.0
4,K. De Bruyne,102000000.0,355000.0


### 3.7 Convert Release Clause to Numerical and Impute NaNs

The 'Release Clause' column, similar to 'Value' and 'Wage', is stored as a string with currency symbols and suffixes (M/K). We will use the same `convert_currency` function to convert it into a numerical format (float).

According to the requirements, missing or invalid 'Release Clause' values should be represented as 0.0. Therefore, after applying the conversion function (which returns NaN for invalid/missing inputs), we will fill any resulting NaN values with 0.0. Finally, we drop the original 'Release Clause' column.

In [20]:
# Apply the currency conversion function
fifa_df['Release_Clause_EUR'] = fifa_df['Release Clause'].apply(convert_currency)

# Impute NaN values with 0.0
fifa_df['Release_Clause_EUR'].fillna(0.0, inplace=True)

# Drop the original 'Release Clause' column
fifa_df.drop(columns=['Release Clause'], inplace=True)

# Verify the changes
print(f"NaNs in Release_Clause_EUR: {fifa_df['Release_Clause_EUR'].isnull().sum()}")
print("Head of 'Release_Clause_EUR':")
display(fifa_df[['Name', 'Release_Clause_EUR']].head())
print("\nDescription of 'Release_Clause_EUR':")
display(fifa_df['Release_Clause_EUR'].describe())

NaNs in Release_Clause_EUR: 0
Head of 'Release_Clause_EUR':


C:\Users\samst\AppData\Local\Temp\ipykernel_21908\3946241328.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fifa_df['Release_Clause_EUR'].fillna(0.0, inplace=True)


,Name,Release_Clause_EUR
0,L. Messi,226500000.0
1,Cristiano Ronaldo,127100000.0
2,Neymar Jr,228100000.0
3,De Gea,138600000.0
4,K. De Bruyne,196400000.0



Description of 'Release_Clause_EUR':


count    1.814700e+04
mean     4.205057e+06
std      1.072275e+07
min      0.000000e+00
25%      3.980000e+05
50%      1.000000e+06
75%      3.000000e+06
max      2.281000e+08
Name: Release_Clause_EUR, dtype: float64

### 3.8 Split Work Rate

The 'Work Rate' column contains two pieces of information (Attacking/Defensive) separated by a '/'. We will split this into two distinct columns, 'AttackingWorkRate' and 'DefensiveWorkRate', for potentially more granular analysis later or use in feature engineering. We will also convert them to categorical types for efficiency.

In [21]:
# Split the 'Work Rate' column
fifa_df[['AttackingWorkRate', 'DefensiveWorkRate']] = fifa_df['Work Rate'].str.split('/ ', expand=True)

# Trim potential leading/trailing whitespace
fifa_df['AttackingWorkRate'] = fifa_df['AttackingWorkRate'].str.strip()
fifa_df['DefensiveWorkRate'] = fifa_df['DefensiveWorkRate'].str.strip()

# Drop the original 'Work Rate' column
fifa_df.drop(columns=['Work Rate'], inplace=True)

# Convert new columns to categorical type (recommended)
fifa_df['AttackingWorkRate'] = fifa_df['AttackingWorkRate'].astype('category')
fifa_df['DefensiveWorkRate'] = fifa_df['DefensiveWorkRate'].astype('category')

# Display value counts to verify
print("Attacking Work Rate Counts:")
display(fifa_df['AttackingWorkRate'].value_counts())
print("\nDefensive Work Rate Counts:")
display(fifa_df['DefensiveWorkRate'].value_counts())

# Display head to see the new columns
display(fifa_df[['AttackingWorkRate', 'DefensiveWorkRate']].head())

Attacking Work Rate Counts:


AttackingWorkRate
Medium    12343
High       4882
Low         922
Name: count, dtype: int64


Defensive Work Rate Counts:


DefensiveWorkRate
Medium    13423
High       3144
Low        1580
Name: count, dtype: int64

,AttackingWorkRate,DefensiveWorkRate
0,Medium,Medium
1,High,Low
2,High,Medium
3,Medium,Medium
4,High,High
